In [ ]:
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from confige import CHROME_DATA_PATH
import os.path
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

#set driver to keep config
os.system("taskkill /im chrome.exe /f")
options = webdriver.ChromeOptions()
options.add_argument(CHROME_DATA_PATH)

#variables
browser = webdriver.Chrome(executable_path='chromedriver.exe', options=options)
dataResult=pd.DataFrame()

#loop to get data in all pages
for i in range(1,12):
   
    #open browser
    page = browser.get(f'https://exames.genera.com.br/busca-parentes?page={i}')
    
    #page load wait
    try:
        myElem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'content-main')))
        print ("Page is ready!")
    except TimeoutException:
        print ("Loading took too much time!")
        
    #create a df with name and email
    nameEmail = pd.DataFrame(columns=['Nome', 'Email'])
    
    for t in range(1,21):
        try:                                                                     #avoid erros on final page with fewer lines
            name = browser.find_element_by_css_selector(f'#realtive-name-{t}').text   #get name
            email = browser.find_element_by_css_selector(f'#realtive-email-{t}').text #get email
            temporaryData = pd.DataFrame([[name,email]], columns=['Nome','Email'])    #create a df with name and email
            nameEmail = pd.concat([nameEmail,temporaryData], ignore_index=True)       #concat name and email to main df
        except:                                                                  
            pass

    #create dataframe and get the table from the webpage
    pageTable = pd.read_html(browser.page_source)[0]

    #remove two columns and insert Name|Email
    tableAdjust= pageTable.drop(['Unnamed: 0', 'Nome'], axis=1)
    tableAdjust.insert(loc=0, column='Email', value=nameEmail['Email'])
    tableAdjust.insert(loc=0, column='Nome', value=nameEmail['Nome'])


    #remove NaN values
    tableAdjust.fillna('-', inplace=True)
    
    #concat values to final df
    dataResult = pd.concat([dataResult,tableAdjust], ignore_index=True)
    
    #confirm loop success
    print(f'Página {i} ok')

#get final df and filter to obtein results
nomeFamilia = dataResult[dataResult.Nome.str.lower().str.contains('garcia|abran|costa|silva')].reset_index(drop=True)

nomeFamilia 